In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# 加载数据
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# 数据预处理
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    x = tf.reshape(x, [28 * 28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y
    

In [4]:
# 划分训练接和测试集
batch_size = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).batch(batch_size)

In [5]:
# 验证数据类型
sample = next(iter(db_train))
print(sample[0].shape, sample[1].shape)

(128, 784) (128, 10)


In [7]:
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])
model.build(input_shape=(None, 28*28))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
model.fit(db_train, epochs=10, validation_data=db_test)

Epoch 1/10
469/469 [==============================] - 7s 15ms/step - loss: 0.5454 - accuracy: 0.7362 - val_loss: 0.4211 - val_accuracy: 0.8490
Epoch 2/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3797 - accuracy: 0.8578 - val_loss: 0.3717 - val_accuracy: 0.8665
Epoch 3/10
469/469 [==============================] - 6s 14ms/step - loss: 0.3395 - accuracy: 0.8743 - val_loss: 0.3469 - val_accuracy: 0.8736
Epoch 4/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3100 - accuracy: 0.8851 - val_loss: 0.3390 - val_accuracy: 0.8766
Epoch 5/10
469/469 [==============================] - 5s 12ms/step - loss: 0.2883 - accuracy: 0.8930 - val_loss: 0.3370 - val_accuracy: 0.8772
Epoch 6/10
469/469 [==============================] - 5s 11ms/step - loss: 0.2720 - accuracy: 0.8997 - val_loss: 0.3372 - val_accuracy: 0.8790
Epoch 7/10
469/469 [==============================] - 6s 12ms/step - loss: 0.2585 - accuracy: 0.9047 - val_loss: 0.3509 - val_accuracy: 0.8767

In [9]:
model.evaluate(db_test)

79/79 [==============================] - 1s 10ms/step - loss: 0.3409 - accuracy: 0.8835


[0.34091750306041935, 0.8835]

### 第一种保存模型的方法:save_weights

In [10]:
# 将模型的权重保存 这个是最简单保存模型的方法
model.save_weights('weights.ckpt')

In [13]:
# 重新建立模型 这个模型与之前的保存的模型必须具有相同的结构
model2 = keras.Sequential([
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])
model2.build(input_shape=(None, 28*28))

model2.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model2.load_weights('weights.ckpt')

In [14]:
model2.evaluate(db_test)

79/79 [==============================] - 1s 13ms/step - loss: 0.3409 - accuracy: 0.8825


[0.34091750306041935, 0.8835]

### 第二种保存模型的方法:save

In [15]:
# 保存模型有关的所有的参数信息及其他信息
model.save('model.h5')

In [16]:
# 重新建立模型 这个模型与之前的保存的模型必须具有相同的结构

model3 = keras.models.load_model('model.h5', compile=False)

model3.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model3.evaluate(db_test)

79/79 [==============================] - 1s 13ms/step - loss: 0.3409 - accuracy: 0.8825


[0.34091750306041935, 0.8835]

### 第三种保存模型的方法:saved_model.save

In [22]:
path = '/tmp/saved_model/'
# 这种保存模式可以跨语言使用 saved_model是一种保存模型的协议
tf.saved_model.save(model, path)


model4 = tf.saved_model.load(path)
# ???  接下来怎么预测数据
f = model4.signatures['serving_default']
f